In [1]:
import numpy as np

# 读取txt文件中的数据
data = np.loadtxt('ZnS.txt')  # 确保文件在当前工作目录或提供完整路径[1,2,3](@ref)

# 提取各列数据
wavelength_um = data[:, 0]  # 第一列：波长（微米）
n_real = data[:, 1]         # 第二列：折射率实部
n_imag = data[:, 2]        # 第三列：折射率虚部

# 计算复折射率 (n + ik)
refractive_index_complex = n_real + 1j * n_imag  # 使用Python内置复数表示[8](@ref)

# 计算复相对介电常数 (ε = ñ² = (n + ik)² = n² - k² + i2nk)
epsilon_complex = refractive_index_complex**2

# 将波长转换为频率
# c = λf => f = c/λ
c = 3e8  # 光速（米/秒）
wavelength_m = wavelength_um * 1e-6  # 将微米转换为米
f = c / wavelength_m  # 计算频率（Hz）

# 创建最终的数据结构
sampledData = [f, epsilon_complex]

# 打印部分结果以供验证
print("频率数组（前5个值）:")
print(f[:5])
print("\n复相对介电常数（前5个值）:")
print(epsilon_complex[:5])

# 如果需要保存处理后的数据
# np.save('processed_data.npy', sampledData)

频率数组（前5个值）:
[1.36363636e+15 1.25000000e+15 1.15384615e+15 1.07142857e+15
 1.00000000e+15]

复相对介电常数（前5个值）:
[7.365165+7.952812j 8.718336+4.50512j  8.165496+3.03863j
 7.755967+2.348544j 7.567627+1.980636j]


In [ ]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import importlib
import math
import random
from scipy.spatial import ConvexHull

#lumapi = importlib.machinery.SourceFileLoader('lumapi', 'E:\\Program Files\\Lumerical\\v241\\api\\python\\lumapi.py').load_module()
lumapi = importlib.machinery.SourceFileLoader('lumapi', 'C:\\Program Files\\Lumerical\\v241\\api\\python\\lumapi.py').load_module()
def basis_function(i, p, knots, t):
    """计算B样条基函数值 (递归实现)"""
    if p == 0:
        return 1.0 if knots[i] <= t < knots[i+1] else 0.0
    
    denom1 = knots[i+p] - knots[i]
    denom2 = knots[i+p+1] - knots[i+1]
    
    term1 = 0.0
    term2 = 0.0
    
    if denom1 != 0:
        term1 = (t - knots[i]) / denom1 * basis_function(i, p-1, knots, t)
    
    if denom2 != 0:
        term2 = (knots[i+p+1] - t) / denom2 * basis_function(i+1, p-1, knots, t)
    
    return term1 + term2

def basis_function_vectorized(i, p, knots, t):
    """向量化的B样条基函数计算 (迭代实现)"""
    N = np.zeros(len(t))
    # 0阶基函数
    N[(knots[i] <= t) & (t < knots[i+1])] = 1.0
    
    # 递归计算高阶基函数
    for k in range(1, p+1):
        # 临时存储当前阶数的基函数值
        N_temp = np.zeros(len(t))
        
        # 计算当前阶数的基函数
        for j in range(i, i+p+1):
            denom1 = knots[j+k] - knots[j]
            denom2 = knots[j+k+1] - knots[j+1]
            
            term1 = 0.0
            term2 = 0.0
            
            if denom1 != 0:
                term1 = (t - knots[j]) / denom1 * N[j-i]
            
            if denom2 != 0:
                term2 = (knots[j+k+1] - t) / denom2 * N[j-i+1]
            
            N_temp[j-i] = term1 + term2
        
        # 更新基函数值
        N = N_temp
    
    return N

def find_span(n, p, knots, t):
    """找到参数t所在的节点区间"""
    # 特殊情况处理
    if t >= knots[n+1]:
        return n
    if t <= knots[p]:
        return p
    
    # 二分查找
    low = p
    high = n+1
    mid = (low + high) // 2
    
    while t < knots[mid] or t >= knots[mid+1]:
        if t < knots[mid]:
            high = mid
        else:
            low = mid
        mid = (low + high) // 2
    
    return mid

def evaluate_nurbs_curve(p, knots, control_points, t_values):
    n = len(control_points) - 1
    curve_points = []
    # print(control_points)
    # print(t_values)
    for t in t_values:
        # 找到t所在的节点区间
        span = find_span(n, p, knots, t)
        
        # 计算非零基函数
        N = np.zeros(p+1)
        for i in range(0, p+1):
            N[i] = basis_function(span-p+i, p, knots, t)
        
        # 计算曲线点
        point = np.zeros(2)
        for i in range(0, p+1):
            idx = span - p + i
            point += N[i] * np.array(control_points[idx])
        
        curve_points.append(point)
    
    return np.array(curve_points)

class Simulation:
    def __init__(self, control_points, degree=2, target_phase=0):
        self.control_points = control_points  # (N,2) 控制点坐标
        self.degree = degree  # NURBS曲线阶数
        self.fdtd = lumapi.FDTD(hide=False)
        self.um = 1e-6
        self.target_phase = target_phase
        self.fdtd.switchtolayout()
        self.phase = 0
        self.fdtd.save("nurbs_thermal_m.fsp")
        ZnS_material = self.fdtd.addmaterial("Sampled data")
        self.fdtd.setmaterial(ZnS_material,"name","ZnS")
        self.fdtd.setmaterial("ZnS","max coefficients",6)
        self.fdtd.setmaterial("ZnS","sampled data",np.transpose(np.array(sampledData)))
        # 生成节点向量
        n = len(control_points) - 1
        m = n + self.degree + 1
        self.knots = np.zeros(m+1)
        
        # 均匀节点向量
        for i in range(0, self.degree+1):
            self.knots[i] = 0.0
            self.knots[m-i] = 1.0
        
        # 内部节点均匀分布
        num_internal = m - 2*(self.degree+1) + 1
        if num_internal > 0:
            for i in range(0, num_internal):
                self.knots[self.degree+1+i] = (i+1) / (num_internal+1)

    def set_control_points(self, control_points):
        self.control_points = control_points
        # 更新节点向量
        n = len(self.control_points[0]) - 1
        m = n + self.degree + 1
        self.knots = np.zeros(m+1)
        # 均匀节点向量
        for i in range(0, self.degree+1):
            self.knots[i] = 0.0
            self.knots[m-i] = 1.0
        
        # 内部节点均匀分布
        num_internal = m - 2*(self.degree+1) + 1
        if num_internal > 0:
            for i in range(0, num_internal):
                self.knots[self.degree+1+i] = (i+1) / (num_internal+1)
        # print("self.knots: ",self.knots)

    def set_target_phase(self, target_phase):
        self.target_phase = target_phase
        
    def setup_simulation(self):
        """基础仿真设置"""
        self.fdtd.switchtolayout()
        self.fdtd.deleteall()

        # 基底设置
        self.fdtd.addrect(name="Ge_top")
        self.fdtd.set("material","Ge (Germanium) - Palik")
        self.fdtd.set('z min',-0.1e-6)
        self.fdtd.set('z max',0)
        self.fdtd.set('x',0)
        self.fdtd.set('x span',70e-6)
        self.fdtd.set('y',0)
        self.fdtd.set('y span',70e-6)

        # 基底设置
        self.fdtd.addrect(name="ZnS_mid")
        self.fdtd.set("material","ZnS")
        self.fdtd.set('z min',-0.7e-6)
        self.fdtd.set('z max',-0.1e-6)
        self.fdtd.set('x',0)
        self.fdtd.set('x span',70e-6)
        self.fdtd.set('y',0)
        self.fdtd.set('y span',70e-6)

        # 基底设置
        self.fdtd.addrect(name="Ge_bottom")
        self.fdtd.set("material","Ge (Germanium) - Palik")
        self.fdtd.set('z min',-0.8e-6)
        self.fdtd.set('z max',-0.7e-6)
        self.fdtd.set('x',0)
        self.fdtd.set('x span',70e-6)
        self.fdtd.set('y',0)
        self.fdtd.set('y span',70e-6)

        # 基底设置
        self.fdtd.addrect(name="Au_bottom")
        self.fdtd.set("material","Au (Gold) - Palik")
        self.fdtd.set('z min',-0.85e-6)
        self.fdtd.set('z max',-0.8e-6)
        self.fdtd.set('x',0)
        self.fdtd.set('x span',70e-6)
        self.fdtd.set('y',0)
        self.fdtd.set('y span',70e-6)

        self.fdtd.addmesh(name="fine_mesh")
        self.fdtd.set('z min',-1e-6)
        self.fdtd.set('z max',0.2e-6)
        self.fdtd.set('x',0)
        self.fdtd.set('x span',70e-6)
        self.fdtd.set('y',0)
        self.fdtd.set('y span',20e-6)
        self.fdtd.set("dx",0.2e-6)
        self.fdtd.set("dy",0.2e-6)
        self.fdtd.set("dz",0.2e-6)

        self.fdtd.addplane(name="source")
        self.fdtd.set('wavelength start',11.6e-6)
        self.fdtd.set('wavelength stop',13.6e-6)
        self.fdtd.set('direction','Backward')
        self.fdtd.set('polarization angle',0)
        self.fdtd.set('z',2e-6)
        self.fdtd.set('x',0)
        self.fdtd.set('y',0)
        self.fdtd.set('x span',70e-6)
        self.fdtd.set('y span',20e-6)
        frequency_points = int(2e-6/(1e-8)) + 2
        # 添加监视器
        self.fdtd.addpower(
            name="R",
            x=0, y=0, z=3e-6,
            x_span=70e-6, y_span=20e-6, 
            monitor_type="2D Z-normal"
        )
        self.fdtd.setglobalmonitor("frequency points",frequency_points)


        zmax =  4e-6
        zmin = -1e-6

        self.fdtd.addfdtd(
            dimension="3D",
            x=0, y=0, z=(zmax+zmin)/2,
            x_span=(12.6e-6)*5, y_span=12.6e-6, z_span=zmax-zmin
        )

        self.fdtd.set("y min bc","Periodic")

        self.fdtd.set("simulation time",5000e-15)

        self.generate_structure(self.control_points)   

    def generate_structure(self, curves):
        um = 1e-6

        period = 12.6 * um
        # 存储所有曲线的顶点矩阵
        all_vertices = []
        self.set_control_points(curves)
        
        for idx, points in enumerate(self.control_points):
            points = np.array(points) * um
            
            # 计算曲线上的点
            num_samples = max(100, 3 * len(points))  # 采样点数
            t_values = np.linspace(0, 1, num_samples)
            t_values = t_values[:-1]
            line_width = 0.5*um + (1-t_values) * 0.5 * um
            # print(t_values)
            # 计算曲线上的点
            center_points = evaluate_nurbs_curve(
                self.degree, 
                self.knots, 
                points, 
                t_values
            )
            # 计算法线方向
            tangents = np.gradient(center_points, axis=0)
            tangents[tangents==0] = 1e-10
            tangents /= np.linalg.norm(tangents, axis=1)[:, np.newaxis]
            normals = np.column_stack((-tangents[:, 1], tangents[:, 0]))
            normals[:,0] =  normals[:,0]* line_width/2
            normals[:,1] =  normals[:,1]* line_width/2
            # 计算多边形顶点
            top_points = center_points + normals
            bottom_points = center_points - normals
            
            # 创建多边形顶点矩阵
            V = np.vstack((
                top_points,
                bottom_points[::-1],
                top_points[0:1]  # 闭合多边形
            ))
            
            all_vertices.append(V)

        # 创建Lumerical脚本
        script = "um = 1e-6;\n"
        script += "period = 12.6 * um;\n"  # 定义周期
        
        array_size = 5  # 5x5阵列
        
        for i, V in enumerate(all_vertices):
            # 为每个原始结构创建阵列
            for row in range(array_size):
                for col in range(array_size):
                    # 计算偏移量
                    x_offset = (col-2) * period
                    y_offset = (row-2) * period
                    
                    # 应用偏移到所有顶点
                    V_offset = V.copy()

                    V_offset[:, 0] += x_offset  # X方向偏移
                    V_offset[:, 1] += y_offset  # Y方向偏移
                    
                    # 转换为Lumerical矩阵格式
                    vertices_str = "[" + ";\n".join(
                        [f"{x}, {y}" for x, y in V_offset]
                    ) + "]"
                    
                    name = f"nurbs_waveguide_{i}_{row}_{col}"
                    script += f"""
                                addpoly;
                                set("name", "{name}");
                                set("x", 0);
                                set("y", 0);
                                set("z", 0);
                                set("z span", 0.05e-6);
                                set("vertices", {vertices_str});
                                set("material", "Ag (Silver) - Johnson and Christy");
                                """
        self.fdtd.eval(script)
    
    def run_forward(self, wavelength_start=8e-6,wavelength_stop=14e-6):
        """运行正向仿真"""
        self.fdtd.switchtolayout()
        self.fdtd.run()
        Reflect = (self.fdtd.getresult('R','T'))["T"]
        phase = np.angle(self.fdtd.getdata('phase','Ex'))
        self.phase = phase

        return Reflect,phase

In [94]:
points2 = np.array([[[-6,0],[-4.5, 10],[-3,-8],[-1.5,8],[0,-4],[1.5,3.5],[3,-3],[4.5,2.5],[6,-1]]])
sim = Simulation(points2)
sim.set_control_points(points2)
sim.setup_simulation()
#Reflect,phase = sim.run_forward(wavelength_start,wavelength_stop)

In [24]:
print(sim.knots)

[0.  0.  0.5 1.  1. ]
